# Get started

## Let's Build the Addin

If you have Autodesk Revit open, close it. The addin need to register a dockable panel on startup!

In [ ]:
dotnet build Jowsy.Revit.KernelAddin -c "Debug R24"

Open Revit and go to the "Add-ins" tab where a button "Show Dockable Pane" will let you show the kernel viewer.

## Let's build the VS Code Polyglot Notebook extension

In [ ]:
# 1. Build the project
dotnet build Jowsy.DotNet.Interactive.Extensions

# Clear any older versions of this extension package from your NuGet cache
#rm ~/.nuget/packages/Jowsy.DotNet.Interactive.Extensions -Force -Recurse -ErrorAction Ignore

dotnet pack Jowsy.DotNet.Interactive.Extensions -c Debug /p:PackageVersion=1.0.0

$nugets = Get-ChildItem -Recurse Jowsy.DotNet.Interactive.Extensions*.nupkg

$nugetPackage = $nugets[0].FullName

In [ ]:
#!set --value @pwsh:nugetPackage --name nugetPackage
var nugetSource = System.IO.Path.GetDirectoryName(nugetPackage);
nugetSource

In [ ]:
#r "nuget:Jowsy.DotNet.Interactive.Extensions"

#i @csharp:nugetSource

## Connecting to the Revit kernel
We use the #!connect magic command to establish connection to the embedded revit kernel. Remember to start the kernel on the Revit side. Goto Addins->Net Interactive->Show Dockable Panel.

In [ ]:
#!connect revit --kernel-name revit24 --revit-version 2024

If connection is established global variables are created for the current UIApplication, UIDocument and Document. Variables are shown in the kernel viewer as seen below. Those variables can be referenced in code cells chosen to run on the Revit kernel.

![](screenshot-variables.png)

In order to get IntelliSense you can import Revit API references. For the moment IntelliSense is not supported when using the revit kernel. A workaround is to uncomment the connector directive to activate the csharp-kernel.

In [ ]:
#r "nuget:Revit.RevitApi.x64, 2023.0.0"
#r "nuget:Revit.RevitApiUi.x64, 2023.0.0"
using Autodesk.Revit.DB;
using Autodesk.Revit.UI;

## Querying the Revit Database
Use display method to output the result from a query. 

NOTE: Objects of type Autodesk.DB.Elements are configured to be rendered as HTML. 

The formatter only recurse to a specific depth (default 3).

In [ ]:
#!revit24
   
   var query = new FilteredElementCollector(doc).OfClass(typeof(Wall))
                                                 .Cast<Wall>();

display(query);


If you just return the variable it will be defined and shown in Variables in the kernel UI in Revit.

In [ ]:
#!revit24
   
   var wall = new FilteredElementCollector(doc).OfClass(typeof(Wall))
                                                 .Cast<Wall>()
                                                 .First();

wall

In [ ]:
Then you can use reference Wall in other cells.

In [ ]:
#!revit24
display(wall.Name);

Level level = doc.GetElement(wall.LevelId) as Level;

display(level.Name);

## Variable Sharing


In [ ]:
#!revit24

//Adapted from https://thebuildingcoder.typepad.com/blog/2015/01/getting-the-wall-elevation-profile.html

// Get the external wall face for the profile
IList<Reference> sideFaces
  = HostObjectUtils.GetSideFaces(wall,
    ShellLayerType.Exterior );
 
Element e2 = doc.GetElement( sideFaces[0] );

Face face = e2.GetGeometryObjectFromReference(
  sideFaces[0] ) as Face;

// The normal of the wall external face.
XYZ normal = face.ComputeNormal( new UV( 0, 0 ) );

// Offset curve copies for visibility.
Transform offset = Transform.CreateTranslation(
  5 * normal );

// If the curve loop direction is counter-
// clockwise, change its color to RED.

Color colorRed = new Color( 255, 0, 0 );

// Get edge loops as curve loops.

IList<CurveLoop> curveLoops
  = face.GetEdgesAsCurveLoops();

// ExporterIFCUtils class can also be used for 
// non-IFC purposes. The SortCurveLoops method 
// sorts curve loops (edge loops) so that the 
// outer loops come first.
IList<IList<CurveLoop>> curveLoopLoop
  = ExporterIFCUtils.SortCurveLoops(
    curveLoops );



In [ ]:
#!revit24   
  var collector = new FilteredElementCollector( doc, uidoc.ActiveView.Id);
 
  var query = collector
            .WhereElementIsNotElementType()
            .WhereElementIsViewIndependent()
            .ToElements();

   var result = query.GroupBy(x => x.Category.Name).Select(y => new {
      Id = y.Key,
      Count = y.Count()
   }).ToList();

   result

In [ ]:
#!set --value @revit24:result --name categoryStats


In [ ]:
#r "nuget: Plotly.NET, 4.2.0"
#r "nuget: Plotly.NET.Interactive, 4.2.0"

In [ ]:
#r "nuget: FSharp.Data, 6.3.0"

In [ ]:
open Plotly.NET
open FSharp.Data
open FSharp.Data.JsonExtensions

let data = JsonValue.Parse(categoryStats.RootElement.GetRawText());

let categories = data.AsArray() |> Seq.map (fun item -> item.GetProperty("Id").AsString())
let counts = data.AsArray() |> Seq.map (fun item -> item.GetProperty("Count").AsInteger())
display(counts)

Chart.Bar(counts, categories)